In [17]:
import numpy as np 
import pandas as pd 
import yfinance as yf 
import datetime as dt 
from _measurements import * 
from _optimization import *
import seaborn as sns 
import matplotlib.pyplot as plt

sns.set_theme()

In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
# List of symbols
symbols = [
    'AAPL', 'MSFT', 'GOOGL', 'AMZN', 'NVDA', 'JNJ', 'WMT', 'KO', 'BAC', 'ADBE', 'CSCO', 'MCD', 'NFLX', 'NKE', 'PUM.DE',
    'PM', 'IBM', 'GE'
]

# Download historical data for the symbols
data = yf.download(symbols, start='2007-01-01', end='2023-08-07')["Close"]

# Print the first few rows of the data
data.dropna(inplace=True)
returns = data.pct_change().dropna()
returns

[*********************100%***********************]  18 of 18 completed


,AAPL,ADBE,AMZN,BAC,CSCO,GE,GOOGL,IBM,JNJ,KO,MCD,MSFT,NFLX,NKE,NVDA,PM,PUM.DE,WMT
Date,,,,,,,,,,,,,,,,,,
2008-03-18,0.048055,0.035401,0.077709,0.082592,0.053108,0.052724,0.045943,0.024751,0.019831,0.029815,0.010501,0.039576,0.060928,0.040668,0.062150,0.004252,0.037614,0.020621
2008-03-19,-0.023716,0.090025,-0.021339,-0.009504,-0.043393,-0.015219,-0.016304,-0.012414,-0.006584,0.009258,-0.021696,-0.027192,-0.003714,0.002595,-0.069056,0.010081,0.002574,-0.003923
2008-03-20,0.027763,-0.008633,0.043038,0.085581,0.012260,0.053386,0.003588,0.011886,0.007707,0.018012,0.014163,0.019567,0.039289,0.087983,0.048698,-0.018962,0.016431,0.048247
2008-03-25,0.057852,0.060668,0.027053,-0.021261,0.039564,-0.005868,0.039742,-0.003042,-0.012695,0.006062,0.032709,-0.001371,0.024282,0.016352,0.097192,0.052899,0.039825,-0.003382
2008-03-26,0.028940,0.018062,-0.018225,-0.027581,-0.037670,-0.003756,0.016438,-0.008985,0.002324,-0.004234,-0.010676,-0.019904,0.019935,-0.030715,-0.031988,-0.028019,-0.017207,-0.002827
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-07-31,0.003166,0.032711,0.011119,0.003135,-0.000960,-0.001311,0.001056,0.005089,-0.039833,-0.008803,-0.002823,-0.007241,0.030978,0.016295,-0.000449,-0.001402,-0.000976,-0.000313
2023-08-01,-0.004276,0.005365,-0.014886,-0.011875,0.011914,-0.008929,-0.008816,-0.005895,0.008118,-0.002584,-0.007265,0.001250,-0.000797,-0.008968,-0.004751,-0.014741,-0.013021,-0.004692
2023-08-02,-0.015490,-0.034238,-0.026426,-0.013599,-0.001519,-0.011040,-0.024097,0.005861,0.006039,0.003076,-0.004982,-0.026283,-0.020336,-0.017276,-0.048122,-0.003155,-0.001979,0.000691


In [28]:
portfo = Portfolio(returns)
portfo.portfolio_stats()
w, _, _ = portfo.optimize_portfolio(objective="max_utility")
w_eq = pd.Series(np.ones(len(returns.columns)) / len(returns.columns), index=returns.columns)
st = pd.DataFrame(Measurements(returns).analyze(returns, w))
st_eq = pd.DataFrame(Measurements(returns).analyze(returns, w_eq))

In [29]:
df = pd.concat([st, st_eq])
df.index=["optimzied_portfolio", "equal_portfolio"]
np.round(df.T, 4)

,optimzied_portfolio,equal_portfolio
cum_return,136.0588,9.4507
max_drawdown,0.5884,0.4434
volatility,0.0220,0.0131
sharpe_ratio,0.0699,0.0537
sortino_ratio,0.0953,0.0674
kurtosis,4.2628,9.4697
skewness,-0.1318,-0.1442
var_hist,0.0335,0.0191
cvar_hist,0.0013,0.0008
var_gauss,0.0346,0.0208


# Log normal , weibull
